Imports the helper functions

In [8]:
from feedback_tool_helper_functions import count_source_lines_chars_helper
from feedback_tool_helper_functions import identifiers_helper
from feedback_tool_helper_functions import calculate_comment
from feedback_tool_helper_functions import count_blocks
from feedback_tool_helper_functions import average_readability_helper
from feedback_tool_helper_functions import log_reg_eqn

Function to extract the necessary features from specified snippet

In [2]:
# Extracts the readability features of a code snippet
# a = "numLines", b = "avgCharsLine", c = "ratioComment", d = "ratioBlank", 
# e = "avgNewId", f = "avgIdLen", g = "ratioIndentNumLines", h = "commentReadabilityFK"
def code_extract(file_path):
    tmp0 = count_source_lines_chars_helper(file_path)
    numLines = tmp0[0]

    a = numLines
    b = tmp0[2]/numLines
    c = calculate_comment(file_path)[1]
    d = tmp0[1]/numLines
    long_lines = tmp0[3]

    tmp1 = identifiers_helper(file_path)
    e = tmp1[2]/numLines
    f = tmp1[5]
    num_Id_in_line = tmp1[6]
    long_Id = tmp1[7]
    
    g = count_blocks(file_path)/numLines
    
    tmp2 = average_readability_helper(file_path)
    h = tmp2[1]
    diff_Comment = tmp2[2]

    return log_reg_eqn(a, b, c, d, e, f, g, h), a, b, c, d, e, f, g, h, long_lines, num_Id_in_line, long_Id, diff_Comment

Feedback Tool

In [3]:
def code_feedback(directory):
    # Extract code features
    code_features = code_extract(directory)
    long_lines = code_features[9]
    num_Id_in_line = code_features[10]
    long_Id = code_features[11]
    diff_Comment = code_features[12]
    
    readability_score = code_features[0]
    code_features = code_features[1:9]
    print("Readability score: " + str(round(readability_score*100, 1)) + "%")
    a, b, c, d, e, f, g, h = code_features
    if (readability_score >= 0.7):
        print("The code you provided is readable, Here's what you did well:")
        print()
        if (a <= 500):
            print("1. Good job on keeping the line count under control. This makes your code easier to read and maintain.")
            print(f"Current number of lines: {a} lines")
            print()

        if (b <= 55):
            print("2. Well done on keeping your lines within an acceptable length. This improves readability.")
            print(f"Current average characters per line: {b:.2f} characters")
            print()

        if (c >= 0.2):
            print("3. Great job on including comments in your code. This helps others understand your code more easily.")
            print(f"Current comment-to-code ratio: {c:.2f}")
            print()

        if (d >= 0.2):
            print("4. Good use of whitespace in your code. This makes your code more visually appealing and easier to read.")
            print(f"Current blank lines-to-code ratio: {d:.2f}")
            print()

        if (e <= 1):
            print("5. Nice work on minimizing the number of new identifiers per line. This helps make your code less complex.")
            print(f"Current average new identifiers per line: {e:.2f}")
            print()

        if (f <= 9):
            print("6. Good job on using concise and clear identifier names. This makes your code easier to understand.")
            print(f"Current average identifier length: {f:.2f} characters")
            print()

        if (g <= 0.5):
            print("7. Your code structure is well-organized, making it easier to follow and understand.")
            print(f"Current number of code in same indentation-to-overall-code ratio: {g:.2f}")
            print()

        if (h >= 75 or h == 0):
            print("8. Your comments are clear and easy to read. Keep up the good work!")
            print(f"Current average Flesch-Kincaid readability score: {h:.2f} (Higher is more readable)")
            return
    else:
        if (readability_score > 0.5):
            print("The code you provided is somewhat readable. Here are some areas to improve:")
        else:
            print("The code you provided is not readable. Here are some suggestions for improvement:")

        print()
        

        if (a > 500):
            print("Your code has too many lines. Consider simplifying your code or splitting it into multiple files to reduce the line count to below 500.")
            print(f"Current number of lines: {a} lines")
            print("Example: If you have a large function, consider breaking it down into smaller, more focused functions.")
            print()

        if (b > 55):
            print(f"Some of your lines are too long. Aim to keep lines under 80 characters.")
            print(f"Current average characters per line: {b:.2f} characters")
            print("Example: Instead of chaining multiple operations on one line, split them across multiple lines.")
            print("These are your long lines:")
            for line in long_lines:
                print("     " + "line " + str(line[0]) + ": " + str(line[1]))
            print()

        if (c < 0.2):
            print(f"Your code could use more comments. Aim for a comment-to-code ratio of at least 1:4.")
            print(f"Current comment-to-code ratio: {c:.2f}")
            print("Example: Write comments to explain the purpose of functions, complex logic, or important variables.")
            print()

        if (d < 0.2):
            print(f"Increase the whitespace in your code to improve readability. Aim for a blank line ratio of at least 0.2")
            print(f"Current blank lines-to-code ratio: {d:.2f}")
            print("Example: Add blank lines between functions or logical sections of your code.")
            print()

        if (e > 1):
            print(f"Too many new identifiers are introduced per line. Simplify your variables and reduce the number of new variables if possible.")
            print(f"Current average new identifiers per line: {e:.2f}")
            print("Example: Group related variables together or use data structures like lists or dictionaries.")
            print("Listed below are lines of your code with more than 2 variable:")
            lineNum = 0
            for line in num_Id_in_line:
                if line > 2:
                    print("line " + str(lineNum))
                lineNum += 1
            print()

        if (f > 9):
            print("Your identifier names are too long. Keep them between 6-9 characters and ensure the names clearly indicate their purpose.")
            print(f"Current average identifier length: {f:.2f} characters")
            print("Example: Instead of 'input_string', use 'in_str' as a shorter, yet stillclear alternative.")
            print("Here are your long identifiers:")
            for id in long_Id:
                print("'" + id + "'")
            print()

        if (g > 0.5):
            print(f"Your code structure could be improved. Organize related segments of code in the same block (same indentation level).")
            print(f"Current number of code in same indentation-to-overall-code ratio: {g:.2f}")
            print("Example: Use consistent indentation to clearly show the structure and nesting of your code.")
            print()

        if (h < 75 and h != 0):
            print(f"Your comments are difficult to read. Write in simple and concise English using easy-to-understand words.")
            print(f"Current average Flesch-Kincaid readability score: {h:.2f} (Higher is more readable)")
            print("Example: Instead of 'This function initializes the variables for the algorithm', write 'Initialize variables for the algorithm'.")
            print("Here are your difficult comments:")
            for comment in diff_Comment:
                print("'" + comment + "'")
            print()

Execution 

In [14]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Construct a relative path to a subdirectory
# Here specify the path to the code you want to test on
subdirectory = "Dataset/Snippets/1.jsnp"
subdirectory_path = os.path.join(current_directory, subdirectory)

# Set Directory Path for snippet
directory_path = subdirectory_path

# Output feedback
code_feedback(directory_path)

Readability score: 80.2%
The code you provided is readable, Here's what you did well:

1. Good job on keeping the line count under control. This makes your code easier to read and maintain.
Current number of lines: 15 lines

2. Well done on keeping your lines within an acceptable length. This improves readability.
Current average characters per line: 25.93 characters

3. Great job on including comments in your code. This helps others understand your code more easily.
Current comment-to-code ratio: 0.21

4. Good use of whitespace in your code. This makes your code more visually appealing and easier to read.
Current blank lines-to-code ratio: 0.27

6. Good job on using concise and clear identifier names. This makes your code easier to understand.
Current average identifier length: 4.86 characters

